In [1]:
# modules import
import pandas as pd

In [2]:
# read csv file
ad_clicks = pd.read_csv('ad_clicks.csv')

In [3]:
# examine few rows
ad_clicks.head()

,user_id,utm_source,day,ad_click_timestamp,experimental_group
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B


In [4]:
# source views count
ad_clicks.groupby(['utm_source']).user_id.count()

utm_source
email       255
facebook    504
google      680
twitter     215
Name: user_id, dtype: int64

In [5]:
# add is_click column
ad_clicks['is_click'] = ~ad_clicks.ad_click_timestamp.isnull()
ad_clicks.head()

,user_id,utm_source,day,ad_click_timestamp,experimental_group,is_click
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A,True
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B,False
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A,False
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B,False
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B,False


In [6]:
# count clicks on ads from each utm_source
clicks_by_source = ad_clicks.groupby(['utm_source', 'is_click']).user_id.count().reset_index()
clicks_by_source

,utm_source,is_click,user_id
0,email,False,175
1,email,True,80
2,facebook,False,324
3,facebook,True,180
4,google,False,441
5,google,True,239
6,twitter,False,149
7,twitter,True,66


In [7]:
# elect only clicked counts
clicks_by_source[clicks_by_source.is_click == True]

,utm_source,is_click,user_id
1,email,True,80
3,facebook,True,180
5,google,True,239
7,twitter,True,66


In [8]:
# clicks pivot table
clicks_pivot = clicks_by_source.pivot(
  columns='is_click',
  index= 'utm_source',
  values= 'user_id'
).reset_index()
clicks_pivot

is_click,utm_source,False,True
0,email,175,80
1,facebook,324,180
2,google,441,239
3,twitter,149,66


In [9]:
# percent of clicks
clicks_pivot['percent_clicked'] = clicks_pivot[True] / (clicks_pivot[True] + clicks_pivot[False])
clicks_pivot

is_click,utm_source,False,True,percent_clicked
0,email,175,80,0.313725
1,facebook,324,180,0.357143
2,google,441,239,0.351471
3,twitter,149,66,0.306977


In [10]:
# experimental ads distribution (percentage)
exp_ads = ad_clicks.groupby('experimental_group').user_id.count().reset_index()
exp_ads['percentage'] = exp_ads.user_id / (exp_ads.user_id.sum())
exp_ads

,experimental_group,user_id,percentage
0,A,827,0.5
1,B,827,0.5


In [11]:
clicks_by_source

,utm_source,is_click,user_id
0,email,False,175
1,email,True,80
2,facebook,False,324
3,facebook,True,180
4,google,False,441
5,google,True,239
6,twitter,False,149
7,twitter,True,66


In [12]:
# which experimental Ad has a higher number of clicks
clicks_experimental = ad_clicks.groupby(['experimental_group', 'is_click']).user_id.count().reset_index()
# pivot table
clicks_exp_pivot = clicks_experimental.pivot(columns='is_click', index='experimental_group', values='user_id').reset_index()
print(clicks_exp_pivot)


is_click experimental_group  False  True
0                         A    517   310
1                         B    572   255


Ad A has a higher perception of clicks for this A/B testing

In [13]:
# Task 9
# create new dataframes a_clicks and b_clicks
a_clicks = ad_clicks[ad_clicks.experimental_group == 'A']
b_clicks = ad_clicks[ad_clicks.experimental_group == 'B']

In [14]:
# Task 10
# grougpby and pivot table Ad A and Ad B
a_clicks = a_clicks.groupby(['is_click', 'day']).user_id.count().reset_index()
a_clicks_pivot = a_clicks.pivot(index='day', columns='is_click', values='user_id').reset_index()

b_clicks = b_clicks.groupby(['is_click', 'day']).user_id.count().reset_index()
b_clicks_pivot = b_clicks.pivot(index='day', columns='is_click', values='user_id').reset_index()

In [15]:
# add new column for clicks percentage of Ad A and Ad B
a_clicks_pivot['percent_clicked'] = a_clicks_pivot[True] / (a_clicks_pivot[True] + a_clicks_pivot[False])
print(a_clicks_pivot)

b_clicks_pivot['percent_clicked'] = b_clicks_pivot[True] / (b_clicks_pivot[True] + b_clicks_pivot[False])
print(b_clicks_pivot)
# compare two tables and come up with conclusions

is_click            day  False  True  percent_clicked
0            1 - Monday     70    43         0.380531
1           2 - Tuesday     76    43         0.361345
2         3 - Wednesday     86    38         0.306452
3          4 - Thursday     69    47         0.405172
4            5 - Friday     77    51         0.398438
5          6 - Saturday     73    45         0.381356
6            7 - Sunday     66    43         0.394495
is_click            day  False  True  percent_clicked
0            1 - Monday     81    32         0.283186
1           2 - Tuesday     74    45         0.378151
2         3 - Wednesday     89    35         0.282258
3          4 - Thursday     87    29         0.250000
4            5 - Friday     90    38         0.296875
5          6 - Saturday     76    42         0.355932
6            7 - Sunday     75    34         0.311927


# Task 11
- Comparing the two dataframes, Ad A has an higher number of clicks by day of the week than Ad B, except for Tuesday.
- Ad A have a higher impact of clicks than Ad B in average
- In conclusion, Ad A is a better choise due to the performance on A/B testing